In [1]:
import unittest
from datetime import date
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from app.services import MovieService
from db.movie_app_db import User, Movie, Mylist, WatchHistory, Genre, Keyword, Studio

import warnings

warnings.filterwarnings('ignore')

class TestMovieService(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        cls.engine = create_engine('sqlite:///:memory:')
        cls.Session = sessionmaker(bind=cls.engine)
        cls.session = cls.Session()
        cls.service = MovieService(cls.session)

        # Create all tables
        # Replace this with actual table creation
        User.metadata.create_all(cls.engine)
        Movie.metadata.create_all(cls.engine)
        Mylist.metadata.create_all(cls.engine)
        WatchHistory.metadata.create_all(cls.engine)
        Genre.metadata.create_all(cls.engine)
        Keyword.metadata.create_all(cls.engine)
        Studio.metadata.create_all(cls.engine)

    @classmethod
    def tearDownClass(cls):
        # Drop all tables
        # Replace this with actual table dropping
        User.metadata.drop_all(cls.engine)
        Movie.metadata.drop_all(cls.engine)
        Mylist.metadata.drop_all(cls.engine)
        WatchHistory.metadata.drop_all(cls.engine)
        Genre.metadata.drop_all(cls.engine)
        Keyword.metadata.drop_all(cls.engine)
        Studio.metadata.drop_all(cls.engine)

    def tearDown(self):      
        result_watch_histories = self.service.read_all_watch_histories()        
        if not result_watch_histories.empty:
            result_watch_histories.apply(lambda row: self.service.delete_watch_history(row['user_id'], row['movie_id']), axis=1)
            
        result_mylists = self.service.read_all_mylists()        
        if not result_mylists.empty:
            result_mylists.apply(lambda row: self.service.delete_mylist(row['user_id'], row['movie_id']), axis=1)
        
        
        
          
        results_movies = self.service.query_by_title(search_string="Inception")
        if not results_movies.empty:
            results_movies['movie_id'].apply(self.service.delete_movie)
            
        result_studios = self.service.read_all_studios()        
        if not result_studios.empty:
            result_studios['studio_id'].apply(self.service.delete_studio)
        
        result_users = self.service.read_all_users()        
        if not result_users.empty:
            result_users['user_id'].apply(self.service.delete_user)            
            
        result_genres = self.service.read_all_genres()        
        if not result_genres.empty:
            result_genres['genre_id'].apply(self.service.delete_genre)
                        
        result_keywords = self.service.read_all_keywords()        
        if not result_keywords.empty:
            result_keywords['keyword_id'].apply(self.service.delete_keyword)    
            
        result_persons = self.service.read_all_persons()        
        if not result_persons.empty:
            result_persons['person_id'].apply(self.service.delete_person)    
            
    
        self.session.rollback()

    

    # -------- User CRUD Tests --------
    def test_create_user(self):
        self.service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        user = self.service.read_user(1)
        self.assertIsNotNone(user)
        self.assertEqual(user.user_name, "JohnDoe")

    def test_read_user(self):
        self.service.create_user(user_name="JaneDoe", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        user = self.service.read_user(1)
        self.assertIsNotNone(user)
        self.assertEqual(user.user_name, "JaneDoe")

    def test_update_user(self):
        self.service.create_user(user_name="OldName", birthdate=date(2000, 1, 1), password="password", e_mail="old@example.com")
        self.service.update_user(1, user_name="NewName", e_mail="new@example.com")
        user = self.service.read_user(1)
        self.assertEqual(user.user_name, "NewName")
        self.assertEqual(user.e_mail, "new@example.com")

    def test_delete_user(self):
        self.service.create_user(user_name="DeleteMe", birthdate=date(2000, 1, 1), password="password", e_mail="delete@example.com")
        self.service.delete_user(1)
        user = self.service.read_user(1)
        self.assertIsNone(user)

    # -------- Mylist CRUD Tests --------
    def test_create_mylist(self):
        mylist = self.service.read_mylist(1, 1)
        self.assertIsNone(mylist)
        self.service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.service.create_movie(movie_name="Inception")
        self.service.create_mylist(1, 1)
        mylist = self.service.read_mylist(1, 1)
        self.assertIsNotNone(mylist)
        self.service.delete_mylist(1, 1)

    def test_read_mylist(self):
        self.service.create_user(user_name="JaneDoe", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        self.service.create_movie(movie_name="Inception")
        self.service.create_mylist(1, 1)
        mylist = self.service.read_mylist(1, 1)
        self.assertIsNotNone(mylist)
        self.service.delete_mylist(1, 1)

    def test_delete_mylist(self):
        self.service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.service.create_movie(movie_name="Inception")
        self.service.create_mylist(1,1)
        self.service.delete_mylist(1,1)
        mylist = self.service.read_mylist(1, 1)
        self.assertIsNone(mylist)

    # -------- WatchHistory CRUD Tests --------
    def test_create_watch_history(self):
        self.service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.service.create_movie(movie_name="Inception")
        self.service.create_watch_history(user_id=1, movie_id=1, watch_date=date(2024, 9, 11), rating=5.0, is_favorite=True)
        watch_history = self.service.read_watch_history(1, 1)
        self.assertIsNotNone(watch_history)
        self.assertEqual(watch_history.rating, 5.0)

    def test_update_watch_history(self):
        self.service.create_user(user_name="JaneDoe", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        self.service.create_movie(movie_name="Inception")
        self.service.create_watch_history(user_id=1, movie_id=1, watch_date=date(2024, 9, 11))
        self.service.update_watch_history(1, 1, rating=4.5, is_favorite=True)
        watch_history = self.service.read_watch_history(1, 1)
        self.assertEqual(watch_history.rating, 4.5)
        self.assertTrue(watch_history.is_favorite)

    def test_delete_watch_history(self):
        self.service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.service.create_movie(movie_name="Inception")
        self.service.create_watch_history(user_id=1, movie_id=1, watch_date=date(2024, 9, 11))
        self.service.delete_watch_history(1, 1)
        watch_history = self.service.read_watch_history(99, 1)
        self.assertIsNone(watch_history)


    # -------- Genre CRUD Tests --------
    def test_create_genre(self):
        self.service.create_genre(genre_name="XXX")
        genre = self.service.read_genre(1)    
        self.assertIsNotNone(genre)
        self.assertEqual(genre.genre_name, "XXX")
    
        
    def test_update_genre(self):
        self.service.create_genre(genre_name="Adventure")
        self.service.update_genre(1, genre_name="Sci-Fi")
        genre = self.service.read_genre(1)
        self.assertEqual(genre.genre_name, "Sci-Fi")
        self.service.delete_genre(1)

    def test_delete_genre(self):
        self.service.create_genre(genre_name="Comedy")
        self.service.delete_genre(1)
        genre = self.service.read_genre(1)
        self.assertIsNone(genre)

    # # -------- Keyword CRUD Tests --------
    def test_create_keyword(self):
        self.service.create_keyword(keyword_name="Sci-Fi")
        keyword = self.service.read_keyword(1)
        self.assertIsNotNone(keyword)
        self.assertEqual(keyword.keyword_name, "Sci-Fi")
        self.service.delete_keyword(1)
            
    def test_update_keyword(self):
        self.service.create_keyword(keyword_name="Thriller")
        self.service.update_keyword(1, keyword_name="Mystery")
        keyword = self.service.read_keyword(1)
        self.assertEqual(keyword.keyword_name, "Mystery")
        self.service.delete_keyword(1)

    def test_delete_keyword(self):
        self.service.create_keyword(keyword_name="Horror")
        self.service.delete_keyword(1)
        keyword = self.service.read_keyword(1)
        self.assertIsNone(keyword)

    # -------- Studio CRUD Tests --------
    def test_create_studio(self):
        self.service.create_studio(studio_name="Marvel Studios")
        studio = self.service.read_studio(1)
        self.assertIsNotNone(studio)
        self.assertEqual(studio.studio_name, "Marvel Studios")
        self.service.delete_studio(1)

    def test_update_studio(self):
        self.service.create_studio(studio_name="Universal")
        self.service.update_studio(1, studio_name="Universal Studios")
        studio = self.service.read_studio(1)
        self.assertEqual(studio.studio_name, "Universal Studios")
        self.service.delete_studio(1)
        
    def test_delete_studio(self):
        self.service.create_studio(studio_name="Paramount")
        self.service.delete_studio(1)
        studio = self.service.read_studio(1)
        self.assertIsNone(studio)

    # -------- Distinct Values Tests --------
    def test_get_distinct_genre_names(self):
        self.service.create_genre(genre_name="Action")
        self.service.create_genre(genre_name="Drama")
        genres = self.service.get_distinct_genre_names()
        self.assertIn("Action", genres)
        self.assertIn("Drama", genres)

    def test_get_distinct_keyword_names(self):
        self.service.create_keyword(keyword_name="Sci-Fi")
        self.service.create_keyword(keyword_name="Action")
        keywords = self.service.get_distinct_keyword_names()
        self.assertIn("Sci-Fi", keywords)
        self.assertIn("Action", keywords)

    def test_get_distinct_person_names(self):
        # Assuming you have a Person model and create_person method
        self.service.create_person(name="John Doe")
        self.service.create_person(name="Jane Doe")
        persons = self.service.get_distinct_person_names()
        self.assertIn("John Doe", persons)
        self.assertIn("Jane Doe", persons)

    def test_get_distinct_studio_names(self):
        self.service.create_studio(studio_name="Warner Bros")
        self.service.create_studio(studio_name="Paramount")
        studios = self.service.get_distinct_studio_names()
        self.assertIn("Warner Bros", studios)
        self.assertIn("Paramount", studios)

# Run the tests in the notebook
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
# if __name__ == '__main__':
#     # Load the test case and the specific test method
#     suite = unittest.TestSuite()
#     suite.addTest(TestMovieService('test_delete_mylist'))
    
#     # Run the test suite
#     runner = unittest.TextTestRunner()
#     runner.run(suite)


..............c:\Users\pogawal\WorkFolder\Documents\Python\cinema_app\app\services.py:191: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  return self.session.query(Person).get(person_id)
.........
----------------------------------------------------------------------
Ran 23 tests in 1.145s

OK


In [ ]:
import unittest
from datetime import date
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from app.services import MovieService
from db.movie_app_db import User

class TestUserService(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        cls.engine = create_engine('sqlite:///:memory:')
        cls.Session = sessionmaker(bind=cls.engine)
        cls.session = cls.Session()
        cls.service = MovieService(cls.session)
        User.metadata.create_all(cls.engine)

    @classmethod
    def tearDownClass(cls):
        User.metadata.drop_all(cls.engine)

    def tearDown(self):
        self.session.rollback()

    def test_create_user(self):
        self.service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        user = self.service.read_user(1)
        self.assertIsNotNone(user)
        self.assertEqual(user.user_name, "JohnDoe")

    # def test_read_user(self):
    #     self.service.create_user(user_name="JaneDoe", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
    #     user = self.service.read_user(1)
    #     self.assertIsNotNone(user)
    #     self.assertEqual(user.user_name, "JaneDoe")

    # def test_update_user(self):
    #     self.service.create_user(user_name="OldName", birthdate=date(2000, 1, 1), password="password", e_mail="old@example.com")
    #     self.service.update_user(1, user_name="NewName", e_mail="new@example.com")
    #     user = self.service.read_user(1)
    #     self.assertEqual(user.user_name, "NewName")
    #     self.assertEqual(user.e_mail, "new@example.com")

    # def test_delete_user(self):
    #     self.service.create_user(user_name="DeleteMe", birthdate=date(2000, 1, 1), password="password", e_mail="delete@example.com")
    #     self.service.delete_user(1)
    #     user = self.service.read_user(1)
    #     self.assertIsNone(user)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
